In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import pandas as pd
import csv
import xml.etree.cElementTree as ET
import xml.dom.minidom
sys.path.append("../")

warnings.filterwarnings("ignore")

In [2]:
from modules.data.bert_data import get_bert_data_loader_for_predict


# parser for xml

In [3]:
class ParseXML_2019(object):
    def __init__(self):
        self.title = []
        self.text = []
        self.all_content = []
        self.header_length = []
        self.entry = ""
        self.entry_node = None
        self.header_node = []
        self.tree = None

    def __get_values(self, root):
        node_list = []
        for node in root:
            node_list.append(node)
        l = len(node_list)
        i = 0
        while i<l:           
            if node_list[i].tag == "section":
                self.__get_values(node_list[i])
            elif node_list[i].tag == "header":
                self.title.append(node_list[i].text)
                self.all_content.append(node_list[i].text + " #header#")
            elif node_list[i].tag == "p":
                if node_list[i-1].tag == "header":
                    self.header_length.append(int(node_list[i-1].text))
                    self.header_node.append(node_list[i-1])
                text = ""
                while node_list[i].tag=="p":
                    text += node_list[i].text
                    i += 1
                    if i>=l:
                        break
                self.text.append(text)
                self.all_content.append(text)
            i += 1
            if i>=l:
                break

    def __get_length(self, root):
        for node in root:
            if node.tag == "p":
                self.summary += (node.text)
                
    def parse(self, xml_path):
        self.title = []
        self.text = []
        self.all_content = []
        self.summary = []
        tree = ET.ElementTree(file=xml_path)
        self.tree = tree
        for node in tree.getroot():
            if node.tag == "title":
                self.entry = node.text
                self.entry_node = node
            elif node.tag == "body":
                self.__get_values(node)
            if node.tag == "summary":
                self.__get_length(node)
        #print self.summary_length

# load model

In [4]:
data_path = "/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/tagged_data/data/"

In [5]:
import torch
torch.cuda.set_device(0)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 0)

In [6]:
from modules.models.bert_models import BertBiLSTMAttnNMT

In [7]:
init_checkpoint_pt = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "pytorch_model.bin")
bert_config_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "bert_config.json")
vocab_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "vocab.txt")

In [8]:
# train a new model
model = BertBiLSTMAttnNMT.create(6,
                                 bert_config_file,
                                 init_checkpoint_pt,
                                 enc_hidden_dim=256,
                                 dec_hidden_dim=256,
                                 dec_embedding_dim=128)

In [9]:
# load a model
model.load_state_dict(
    torch.load(
        "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/models/multiling-2019/BertBiLSTMAttnNMT.cpt"
    ))

# make dataset(only use testset) and load learner

In [10]:
from modules import BertNerData as NerData

In [11]:
train_path = data_path + "train.csv"
valid_path = data_path + "valid.csv"

In [12]:
data = NerData.create(train_path, valid_path, vocab_file, batch_size=8)

In [13]:
from modules import NerLearner

In [14]:
sup_labels = ['B_MISC', 'I_MISC']

In [15]:
num_epochs = 10

In [16]:
learner = NerLearner(
    model,
    data,
    best_model_path=
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/models/multiling-2019/BertBiLSTMAttnNMT.cpt",
    lr=0.01,
    clip=1.0,
    sup_labels=sup_labels,
    t_total=num_epochs * len(data.train_dl))

# make title test set

In [17]:
def make_test_csv(path,para=None):
    if para==None:
        para = open(path[:-4] + ".txt", "r").read()
    data_tag = []
    with open(path[:-4] + ".csv", 'w', encoding='utf-8', newline='') as f:
        f_csv = csv.writer(f)
        if "zh" in path:
            for line in para.split("。"):
                l = len(line)
                line = str(line)
                #                 tag = ["O" for w in range(l)]
                data_tag.append([" ".join(["O" for w in line]), " ".join([w for w in line])])
        else:
            for line in para.split(". "):
                l = len(line.split(" "))
                tag = ["O" for _ in range(l)]
                data_tag.append([" ".join(tag), line])
        f_csv.writerows(data_tag)
    return

# pick title based on tagged results and choose one that best suits length

In [32]:
def show_result(f, preds, title_len, zh):
    result = ""
    max_dis = 32768
    best_sentence = ""
    for sentence, pred in zip(f, preds):
        for i in range(len(pred)):
            if pred[i] == 'B_MISC':
                answer = sentence.bert_tokens[i]
                for j in range(i + 1, len(pred)):
                    if pred[j] == 'I_MISC':
                        answer += " " + sentence.bert_tokens[j]
                    else:
                        break
                len_answer = len(answer)
                if abs(len_answer - title_len) < max_dis:
                    max_dis = abs(len_answer - title_len)
                    best_sentence = sentence
                    if zh:
                        result = "".join(answer.split(" "))
                    else:
                        result = answer
#     if result != "":
#         if zh:
#             print("".join(best_sentence.bert_tokens))
#         else:
#             print(" ".join(best_sentence.bert_tokens))
    print("@@" + result + "@@",end=' ')
    return result

# choose a language and set relative paths

In [20]:
path_result = "/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/test/MultiLing2019-Summary-2length/result/"

In [21]:
path_head = "/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/test/"

In [22]:
lan_path = path_head + "MultiLing2019-Summary-2length/2019.07.06-10.54.47-SentenceVec/"

In [34]:
def make_title(test_path, original_path, zh, language):
    for article in os.listdir(test_path):
        if "csv" in article and "subtitle" not in article:
#             print(article)
            tree = ET.ElementTree(file=original_path + article[:-4] + ".xml")
            root = tree.getroot()
            title_len = int(root[0].text.lower())
#             print(title_len)
            dl, f = get_bert_data_loader_for_predict(test_path + article, learner)
            preds = learner.predict(dl)
            parser = ParseXML_2019()        
            parser.parse(original_path + article[:-4] + ".xml")
            parser.entry_node.text = show_result(f, preds, title_len, zh)
            parser.tree.write(path_result + language + "/" + article[:-4] + ".xml", encoding="utf-8",xml_declaration=True)

In [37]:
def make_subtitle(test_path, original_path, zh, language):
    for file_name in os.listdir(original_path):
        if "xml" in file_name:
            print(file_name)
            parser = ParseXML_2019()     
            try:
                parser.parse(path_result + language + "/" + file_name[:-4] + ".xml")
        #         print(len(parser.header_node),len(parser.header_length),len(parser.text))
                for length,text,header in zip(parser.header_length, parser.text, parser.header_node):
                    tmp_csv_for_subtitle = test_path + str(file_name[:-4]) + "_subtitle"
                    make_test_csv(tmp_csv_for_subtitle + ".xml", text) 
                    try:
                        dl, f = get_bert_data_loader_for_predict(tmp_csv_for_subtitle + ".csv", learner)
                    except RuntimeError:
                        print(file_name)
                    preds = learner.predict(dl)
                    header.text = show_result(f, preds, length, zh)
                parser.tree.write(path_result + language + "/" + file_name[:-4] + ".xml", encoding="utf-8",xml_declaration=True)
            except FileNotFoundError:
                print(file_name)

In [39]:
for language in os.listdir(lan_path):
    if "txt" not in language and "ja" in language:
        print("\n-------------------------" + language + "----------------------------\n")
        test_path = path_head + "MultiLing2019-Summary-2length/2019.07.06-10.54.47-SentenceVec/" + language + "/systems/"
        original_path = path_head + "WikiTest19/clipped/" + language + "/"
        if "zh" in test_path or "ja" in test_path:
            zh = True
        else:
            zh = False
        for idx, f in enumerate(os.listdir(test_path)):
            if "txt" in f:
#                 print(idx, f)
                make_test_csv(test_path + f)
        make_title(test_path, original_path, zh, language)
        make_subtitle(test_path, original_path, zh, language)        


-------------------------ja----------------------------

@@下@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@オ@@ @@エ@@ @@##松@@ @@@@ @@@@ @@旧@@ @@@@ @@@@ @@ワ@@ @@@@ @@ス@@ @@@@ @@北@@ @@@@ @@@@ @@@@ 9e833ce5d08c4699e9f31a95f547c99d.xml
@@@@ @@@@ @@山@@ @@@@ @@@@ @@ボ@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@海@@ @@@@ @@下@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@工@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@@@ @@国@@ @@@@ @@@@ @@@@ @@@@ 8703a9f6344124436ead15d79e32bc18.xml
@@三@@ @@千@@ @@下@@ @@@@ @@@@ 

KeyboardInterrupt: 

In [ ]:
language = "zh"

test_path = path_head + "MultiLing2019-Summary-2length/2019.07.06-10.54.47-SentenceVec/" + language + "/systems/"
original_path = path_head + "WikiTest19/clipped/" + language + "/"

# make test csv for title

In [ ]:
for idx, f in enumerate(os.listdir(test_path)):
    if "txt" in f:
        print(idx, f)
        make_test_csv(test_path + f)

# show results for title

In [ ]:
if "zh" in test_path:
    zh = True
else:
    zh = False
for article in os.listdir(test_path):
    if "csv" in article and "subtitle" not in article:
        print(article)
        tree = ET.ElementTree(file=original_path + article[:-4] + ".xml")
        root = tree.getroot()
        title_len = int(root[0].text.lower())
        print(title_len)
        dl, f = get_bert_data_loader_for_predict(test_path + article, learner)
        preds = learner.predict(dl)
        parser = ParseXML_2019()        
        parser.parse(original_path + article[:-4] + ".xml")
        parser.entry_node.text = show_result(f, preds, title_len, zh)
        parser.tree.write(test_path + article[:-4] + "_result.xml", encoding="utf-8",xml_declaration=True)

# make test csv for subtitles

In [ ]:
for file_name in os.listdir(original_path):
    if "xml" in file_name:
        print(file_name)
        parser = ParseXML_2019()        
        parser.parse(test_path + file_name[:-4] + "_result.xml")
#         print(len(parser.header_node),len(parser.header_length),len(parser.text))
        for length,text,header in zip(parser.header_length, parser.text, parser.header_node):
            tmp_csv_for_subtitle = test_path + str(file_name[:-4]) + "_subtitle"
            make_test_csv(tmp_csv_for_subtitle + ".xml", text) 
            dl, f = get_bert_data_loader_for_predict(tmp_csv_for_subtitle + ".csv", learner)
            preds = learner.predict(dl)
            header.text = show_result(f, preds, length, zh)
        parser.tree.write(test_path + file_name[:-4] + "_result.xml", encoding="utf-8",xml_declaration=True)